In [1]:
import sqlite3, os, sys
import pandas as pd
import tensorflow as tf
import numpy as np
from tqdm.notebook import tqdm

In [2]:
db_path = "../db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db"

In [3]:
conn = sqlite3.connect(db_path)

In [4]:
df_feat=pd.read_sql("select * from features limit 100000", conn)

In [5]:
for col in df_feat.columns:
    if col not in ['row', 'event_no']:
        print(col, np.unique(df_feat[col], return_counts=True))

string (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86], dtype=int64), array([ 799,  741,  682,  688,  726,  807,  738,  785,  779,  782,  738,
        694,  769,  872,  837,  718,  831,  769,  683,  725,  750,  768,
        780,  744,  879, 1176, 1210,  717,  679,  649,  771,  794,  823,
        710, 1117, 2143, 1021,  725,  710,  669,  705,  750,  820,  836,
       1376, 1232,  826,  806,  678,  675,  686,  792,  833,  798,  800,
        805,  750,  745,  690,  770,  735,  818,  777,  704,  756,  758,
        753,  830,  689,  859,  680,  773,  801,  683,  702,  781,  790,
        763, 4390, 4735, 4457, 4018, 4686, 5124, 5008, 4559], dtype=int64))
dom 

In [6]:
df_truth=pd.read_sql("select * from truth limit 100", conn)

In [7]:
n_data = 100
start_id = conn.execute(f"select distinct event_no from truth where pid = 13")
stop_id  = conn.execute(f"select distinct event_no from truth where pid = 13 limit 1 offset {n_data}")

In [30]:
diff={'reg':["l1","l2", None],
     'glob':[True, False]}
len(diff.keys()), list(diff.keys())[0]

(2, 'reg')

In [31]:
keys=list(diff.keys())
len(keys), keys[0]

(2, 'reg')

In [8]:
start_id.fetchall()

[(138674340,),
 (138674341,),
 (138674342,),
 (138674343,),
 (138674344,),
 (138674345,),
 (138674346,),
 (138674347,),
 (138674348,),
 (138674351,),
 (138674352,),
 (138674353,),
 (138674354,),
 (138674355,),
 (138674358,),
 (138674359,),
 (138674360,),
 (138674361,),
 (138674362,),
 (138674363,),
 (138674364,),
 (138674366,),
 (138674367,),
 (138674368,),
 (138674369,),
 (138674370,),
 (138674372,),
 (138674374,),
 (138674375,),
 (138674376,),
 (138674377,),
 (138674378,),
 (138674382,),
 (138674383,),
 (138674384,),
 (138674385,),
 (138674386,),
 (138674388,),
 (138674389,),
 (138674390,),
 (138674391,),
 (138674392,),
 (138674393,),
 (138674394,),
 (138674395,),
 (138674396,),
 (138674398,),
 (138674399,),
 (138674400,),
 (138674401,),
 (138674403,),
 (138674404,),
 (138674405,),
 (138674406,),
 (138674409,),
 (138674410,),
 (138674412,),
 (138674413,),
 (138674414,),
 (138674415,),
 (138674417,),
 (138674418,),
 (138674420,),
 (138674421,),
 (138674422,),
 (138674424,),
 (13867442

In [9]:
stop_id.fetchall()

[(138674472,)]

In [10]:
start, stop = 0, 100000
# start_id  = conn.execute(f"select distinct event_no from features where pid = 13 limit ")
event_nos = pd.read_sql(f"select distinct event_no from features limit {stop} offset {start}", conn)

In [11]:
event_nos

,event_no
0,120102888
1,120102889
2,120102890
3,120102891
4,120102892
...,...
99995,120228027
99996,120228028
99997,120228029
99998,120228030


In [12]:
start, stop = 0, 100000
select_rows = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]

start_id = conn.execute(f"select distinct event_no from features limit 1 offset {start}").fetchall()[0][0]
stop_id  = conn.execute(f"select distinct event_no from features limit 1 offset {stop}").fetchall()[0][0]

In [13]:
features = ["dom_x", "dom_y", "dom_z", "time", "charge_log10"]
targets  = ["energy_log10", "position_x", "position_y", "position_z", "direction_x", "direction_y", "direction_z"]
feature_call = ", ".join(features)
target_call  = ", ".join(targets)
df_event = pd.read_sql(f"select event_no       from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_feat  = pd.read_sql(f"select {feature_call} from features where event_no >= {start_id} and event_no < {stop_id}", conn)
df_targ  = pd.read_sql(f"select {target_call } from truth    where event_no >= {start_id} and event_no < {stop_id}", conn)

In [14]:
idx_list    = np.array(df_event)
x_not_split = np.array(df_feat)

In [15]:
np.shape(idx_list)

(5965924, 1)

In [16]:
ids, idx, counts = np.unique(idx_list.flatten(), return_index = True, return_counts = True) 

In [17]:
np.sum(counts)

5965924

In [18]:
x_not_split.shape

(5965924, 5)

In [19]:
xs = np.split(x_not_split, np.cumsum(counts))

In [20]:
ys = np.array(df_targ)

In [21]:
np.shape(xs)

C:\Users\chris\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(100001,)

In [22]:
for x, y in zip(xs, ys):
    print(x, y)
    break

[[-1.84428951 -1.22482404  0.39792449 -1.56428307  0.16261479]
 [-0.2078301  -0.18691354 -0.44418075 -1.68247499 -1.92155384]
 [ 0.06322446 -0.29591152 -0.46829432 -0.05335309 -0.38592265]
 [ 0.06322446 -0.29591152 -0.14411839  0.90848462  0.23806267]
 [ 0.06322446 -0.29591152 -0.14411839 -1.41830308  1.49374495]
 [ 0.06322446 -0.29591152 -0.08772599 -0.02074843 -1.65692291]
 [ 0.06322446 -0.29591152 -0.08772599 -0.03223416  0.44497857]
 [ 0.          0.29478052 -0.12298133  0.44979622 -0.38592265]
 [-0.04194698 -0.15939176 -0.25229772 -0.30048166  0.        ]
 [-0.04194698 -0.15939176 -0.23821973 -0.30085217  0.89259397]
 [-0.04194698 -0.15939176 -0.2241216  -0.29529455 -1.0557719 ]
 [-0.04194698 -0.15939176 -0.1818273  -0.26083735 -0.62103541]
 [-0.07899002  2.28041226  1.09275381  1.07373101  0.50835111]
 [ 0.74159036  1.95723031  0.97476017  1.00629863  0.68454405]
 [-0.25763961  1.79339814  0.32990768  0.42645424  0.31007972]
 [-0.25763961  1.79339814  0.32990768 -1.44942571  0.08

In [23]:
from sklearn.neighbors import kneighbors_graph as knn
from spektral.data import Graph
neighbors=6

In [24]:
print("Generating adjacency matrices")
graph_list = []
for x, y in tqdm(zip(xs, ys), total = len(xs)):
    a = knn(x[:, :3], neighbors)



    graph_list.append(Graph(x = x, a = a, y = y))

graph_list = np.array(graph_list, dtype = object)

Generating adjacency matrices


KeyboardInterrupt: 

In [ ]:
s = [x.shape[0] for x in xs]

In [ ]:
xs = np.array(xs, dtype = object)[np.argsort(s)]

In [ ]:
size, idx = np.unique(np.sort(s), return_index = True)

In [ ]:
preprocess(list(xs[4: 9]))

In [ ]:
np.sort(s)

In [ ]:
idx

In [ ]:
import pickle
transformers = pickle.load(open('../db_files/muongun/transformers.pkl', 'rb'))
trans_x      = transformers['features']
trans_y      = transformers['truth']

In [ ]:
help(transformers['features']['dom_x'])